In [ ]:
!pip install sentencepiece
!pip install mecab-ko-dic
!pip install mecab-python3
!pip install mecab-ko

In [ ]:
#라이브러리 임포팅
import pandas as pd
import numpy as np
import re
import sentencepiece as spm
import mecab

In [4]:
#데이터 준비
train_data = pd.read_csv('ChatbotData.csv') 
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [29]:
#데이터 전처리
#한글, 숫자, 특수문자를 제외한 모든 문자 제거 및 소문자화

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 대문자를 소문자로 변환
    sentence = re.sub(r"([?.!,¿\\\"'():;-])", r" \1 ", sentence)
    #  2 특수문자 양쪽에 공백 추가
    sentence = re.sub(r' {2,}', ' ', sentence) # 둘 이상의 공백을 하나의 공백으로 치환
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9?.!,]+", " ", sentence) #한글 숫자 특수문자만 남김
    sentence = sentence.strip() # 문자열 양 끝 공백 제거
    return sentence

    
questions = []
for sentence in train_data['Q']:
    sentence = preprocess_sentence(sentence)
    questions.append(sentence)

answers = []
for sentence in train_data['A']:
    sentence = preprocess_sentence(sentence)
    answers.append(sentence)

In [6]:
with open('all.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(questions))
    f.write('\n'.join(answers))

In [14]:
corpus = "all.txt"
prefix = "chatbot"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=all.txt --model_prefix=chatbot --vocab_size=8007 --model_type=bpe --max_sentence_length=999999 --pad_id=0 --pad_piece=[PAD] --unk_id=1 --unk_piece=[UNK] --bos_id=2 --bos_piece=[BOS] --eos_id=3 --eos_piece=[EOS] --user_defined_symbols=[SEP],[CLS],[MASK]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: all.txt
  input_format: 
  model_prefix: chatbot
  model_type: BPE
  vocab_size: 8007
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: [SEP]
  user_defined_sym

In [ ]:
#sentencePieceProcessor가 정상적으로 수행되는지 확인
vocab_file = "../../../../../Downloads/chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "안녕하세요 만나서 반갑습니다"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)

print(line)
print(pieces)
print(ids)

In [6]:
#Lexicla Aug를 이용한 데이터 증강
import random
import gensim

wv = gensim.models.KeyedVectors.load_word2vec_format("ko_c.bin", binary=True)

def lexical_augmentation(sentence_list, wv_model, num_augs=1):
    """
    주어진 문장 리스트에 대해 Lexical Substitution을 적용하여 데이터를 증강하는 함수.

    Args:
        sentence_list (list): 증강할 원본 문장들의 리스트.
        wv_model: Gensim의 KeyedVectors 객체.
        num_augs (int): 원본 문장 1개당 생성할 증강 문장의 개수.

    Returns:
        list: 원본 문장과 증강된 문장이 모두 포함된 리스트.
    """

    # (내부 lexical_sub 함수는 이전과 동일)
    def lexical_sub(sentence, wv):
        tokens = sentence.split()
        valid_tokens = [tok for tok in tokens if tok in wv.key_to_vec]
        if not valid_tokens:
            return None # 변경: 증강 실패 시 None 반환
        selected_tok = random.choice(valid_tokens)
        try:
            similar_word = wv.most_similar(selected_tok)[0][0]
        except KeyError:
            return None # 변경: 증강 실패 시 None 반환
        return " ".join([similar_word if tok == selected_tok else tok for tok in tokens])


    augmented_corpus = []
    print(f"데이터 증강을 시작합니다 (문장당 최대 {num_augs}개 생성)...")

    # 모든 원본 문장을 우선 корпу스에 포함시킵니다.
    augmented_corpus.extend(sentence_list)

    for sentence in tqdm(sentence_list):
        for _ in range(num_augs):
            new_sentence = lexical_sub(sentence, wv_model)
            if new_sentence: # 증강에 성공한 경우에만 추가
                augmented_corpus.append(new_sentence)

    # 최종 중복 제거
    augmented_corpus = list(dict.fromkeys(augmented_corpus))
    print(f"데이터 증강 완료! (원본: {len(sentence_list)} 문장 -> 증강 후: {len(augmented_corpus)} 문장)")

    return augmented_corpus

# --- 사용 예시 ---
# que_train_str, ans_train_str가 있다고 가정
# 원본의 약 2배로 증강 (원본 1 + 증강 1)
# que_train_augmented = lexical_augmentation(que_train_str, wv, num_augs=1)
# ans_train_augmented = lexical_augmentation(ans_train_str, wv, num_augs=1)

# 원본의 약 3배로 증강 (원본 1 + 증강 2)
# que_train_augmented = lexical_augmentation(que_train_str, wv, num_augs=2)
# ans_train_augmented = lexical_augmentation(ans_train_str, wv, num_augs=2)

In [ ]:
from sklearn.model_selection import train_test_split
#train, test 분리
que_train, que_test, ans_train, ans_test = train_test_split(
                                                    questions,
                                                    answers,
                                                    test_size = 0.25,
                                                    shuffle = True,
                                                    random_state = 21)


# 4. 전처리된 train데이터를 Lexical aug를 이용해 증강
if 'wv' in globals() and wv is not None:
    que_train_augmented = lexical_augmentation(que_train, wv, num_augs=2)
    ans_train_augmented = lexical_augmentation(ans_train, wv, num_augs=2)
    # 질문(Question) 학습 데이터 증강
    #que_train = lexical_augmentation(que_train, wv)

    # 답변(Answer) 학습 데이터 증강
    #ans_train = lexical_augmentation(ans_train, wv)

In [ ]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

START_TOKEN = [2]
END_TOKEN = [3]

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    zeros1 = np.zeros(MAX_LENGTH, dtype=int)
    zeros2 = np.zeros(MAX_LENGTH, dtype=int)
    sentence1 = START_TOKEN + vocab.encode_as_ids(sentence1) + END_TOKEN
    zeros1[:len(sentence1)] = sentence1[:MAX_LENGTH]

    sentence2 = START_TOKEN + vocab.encode_as_ids(sentence2) + END_TOKEN
    zeros2[:len(sentence2)] = sentence2[:MAX_LENGTH]

    tokenized_inputs.append(zeros1)
    tokenized_outputs.append(zeros2)
  return tokenized_inputs, tokenized_outputs

In [ ]:
#인코딩 되어 잘 저장된것을 확인
questions_encode, answers_encode = tokenize_and_filter(questions, answers)
print(questions_encode[0])
print(answers_encode[0])

In [ ]:
def encode_and_pad(corpus, vocab, max_len):
    encoded_corpus = []
    for sentence_tokens in corpus:
        # 1. 토큰 리스트를 정수 ID 리스트로 변환
        ids = [vocab.piece_to_id(token) for token in sentence_tokens]

        # 2. 시작 토큰과 종료 토큰 추가
        ids = START_TOKEN + ids + END_TOKEN

        # 3. 최대 길이에 맞춰 패딩
        padded_ids = np.zeros(max_len, dtype=int)
        padded_ids[:len(ids)] = ids[:max_len] # 최대 길이를 넘으면 자르기

        encoded_corpus.append(padded_ids)

    return np.array(encoded_corpus)

In [54]:
def build_corpus(src_data, tgt_data, tokenizer, max_len=40):
    """
    소스 및 타겟 데이터를 받아 전처리 후 토큰화된 말뭉치를 생성합니다.

    Args:
        src_data (list): 소스 문장 리스트
        tgt_data (list): 타겟 문장 리스트
        tokenizer (function): 사용할 토크나이저 함수 (e.g., mecab.morphs)
        max_len (int): 문장의 최대 토큰 길이

    Returns:
        tuple: 정제되고 중복이 제거된 (소스 말뭉치, 타겟 말뭉치)
    """
    corpus_pairs = []
    for src_sentence, tgt_sentence in zip(src_data, tgt_data):
        # 2-1. 문장 정제
        src_preprocessed = preprocess_sentence(src_sentence)
        tgt_preprocessed = preprocess_sentence(tgt_sentence)

        # 2-2. 토큰화
        src_tokens = tokenizer.encode_as_pieces(src_preprocessed) # 객체의 메소드를 호출
        tgt_tokens = tokenizer.encode_as_pieces(tgt_preprocessed)

        # 3. 토큰 개수 필터링
        if len(src_tokens) > max_len or len(tgt_tokens) > max_len:
            continue

        corpus_pairs.append((" ".join(src_tokens), " ".join(tgt_tokens)))

    # 4. 중복 제거
    # 데이터프레임을 사용하여 소스-타겟 쌍을 유지하면서 중복을 안전하게 제거
    df = pd.DataFrame(corpus_pairs, columns=['source', 'target'])

    # 소스 문장 기준 중복 제거 (첫 번째 등장 쌍 유지)
    df = df.drop_duplicates(subset='source', keep='first')

    # 타겟 문장 기준 중복 제거 (첫 번째 등장 쌍 유지)
    df = df.drop_duplicates(subset='target', keep='first')

    # 다시 토큰 리스트 형태로 변환하여 반환
    cleaned_src_corpus = [s.split() for s in df['source']]
    cleaned_tgt_corpus = [t.split() for t in df['target']]

    print("데이터 준비 끝!")
    return cleaned_src_corpus, cleaned_tgt_corpus

In [ ]:

# 4. 함수를 활용하여 que_corpus, ans_corpus 생성
# 기존 CSV 파일의 'Q', 'A' 컬럼을 리스트로 변환하여 사용
from sklearn.model_selection import train_test_split
questions_list = list(train_data['Q'])
answers_list = list(train_data['A'])

que_corpus, ans_corpus = build_corpus(questions_list, answers_list, vocab, max_len=40)


que_corpus_train, que_corpus_test, ans_corpus_train, ans_corpus_test = train_test_split(que_corpus,
                                                    ans_corpus,
                                                    test_size = 0.25,
                                                    shuffle = True,
                                                    random_state = 21)

que_corpus_train_encoded = encode_and_pad(que_corpus_train, vocab, MAX_LENGTH)
ans_corpus_train_encoded = encode_and_pad(ans_corpus_train, vocab, MAX_LENGTH)

que_corpus_test_encoded = encode_and_pad(que_corpus_test, vocab, MAX_LENGTH)
ans_corpus_test_encoded = encode_and_pad(ans_corpus_test, vocab, MAX_LENGTH)


모델링 시작


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, questions, answers):
        self.inputs = questions
        self.dec_inputs = answers[:,:-1]
        self.outputs = answers[:,1:]
        self.length = len(questions)
    
    def __getitem__(self,idx):
        return (self.inputs[idx], self.dec_inputs[idx], self.outputs[idx])

    def __len__(self):
        return self.length

# 3. 전처리된 데이터를 사용하여 DataLoader 생성
BATCH_SIZE = 64
train_dataset = SequenceDataset(que_corpus_train_encoded, ans_corpus_train_encoded)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)



# (선택) 테스트용 DataLoader도 만들 수 있습니다.
test_dataset = SequenceDataset(que_corpus_test_encoded, ans_corpus_test_encoded)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)

print("DataLoader 준비 완료!")

In [ ]:
from torch.nn import Transformer
from torch import nn
import torch
import math

class TFModel(nn.Module):
    #ntoken: vocab의 size
    #ninp: embedding할 차원의 크기
    #nhead: num head
    #nhid: feedforward의 차원
    #nlayers: layer의 개수
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TFModel, self).__init__()
        self.transformer = Transformer(ninp, nhead, dim_feedforward=nhid, num_encoder_layers=nlayers, num_decoder_layers=nlayers,dropout=dropout)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.encoder = nn.Embedding(ntoken, ninp)

        self.pos_encoder_d = PositionalEncoding(ninp, dropout)
        self.encoder_d = nn.Embedding(ntoken, ninp)

        self.ninp = ninp
        self.ntoken = ntoken

        self.linear = nn.Linear(ninp, ntoken)
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)

        tgt = self.encoder_d(tgt) * math.sqrt(self.ninp)
        tgt = self.pos_encoder_d(tgt)


        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

In [63]:
device = torch.device("cuda")

lr = 1e-4
model_aug_v1 = TFModel(vocab_size+7, 256, 8, 512, 2, 0.2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

/opt/conda/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [64]:
epoch = 30
from tqdm import tqdm

model_aug_v1.train()
for i in range(epoch):
    batchloss = 0.0
    progress = tqdm(dataloader)
    for (inputs, dec_inputs, outputs) in progress:
        optimizer.zero_grad()
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device)
        src_padding_mask = gen_attention_mask(inputs).to(device)
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)
        tgt_padding_mask = gen_attention_mask(dec_inputs).to(device)

        result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
        loss = criterion(result.permute(1,2,0), outputs.to(device).long())
        progress.set_description("{:0.3f}".format(loss))
        loss.backward()
        optimizer.step()
        batchloss += loss
    print("epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(dataloader))

  0%|          | 0/185 [00:00<?, ?it/s]/opt/conda/lib/python3.12/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
0.526: 100%|██████████| 185/185 [00:09<00:00, 20.03it/s]


epoch: 1 | loss: 0.4821202458562078


0.485: 100%|██████████| 185/185 [00:09<00:00, 19.86it/s]


epoch: 2 | loss: 0.43904361209353887


0.397: 100%|██████████| 185/185 [00:09<00:00, 20.27it/s]


...

epoch: 28 | loss: 0.007799176267675451


0.008: 100%|██████████| 185/185 [00:08<00:00, 20.70it/s]


epoch: 29 | loss: 0.0074699872248881576


0.005: 100%|██████████| 185/185 [00:08<00:00, 20.70it/s]


epoch: 30 | loss: 0.006289432499859784


In [70]:
def model_train(model) :
    model.train()
    
    for i in range(model, version):
        batchloss = 0.0
        progress = tqdm(dataloader)
        for (inputs, dec_inputs, outputs) in progress:
            optimizer.zero_grad()
            src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device)
            src_padding_mask = gen_attention_mask(inputs).to(device)
            tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)
            tgt_padding_mask = gen_attention_mask(dec_inputs).to(device)
    
            result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
            loss = criterion(result.permute(1,2,0), outputs.to(device).long())
            progress.set_description("{:0.3f}".format(loss))
            loss.backward()
            optimizer.step()
            batchloss += loss
        print("epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(dataloader))

    #모델 저장 및 로드
    PATH = "chatbot_model_state_aug_v{}.pth".format(version)
    
    torch.save(model.state_dict(), PATH)
    print(f"모델이 {PATH} 경로에 저장되었습니다.")

In [ ]:
model_train()

In [ ]:
#저장된 모델 로드
loaded_model = TFModel(vocab_size+7, 256, 8, 512, 2, 0.2).to(device)

PATH = "chatbot_model_state_dict.pth"
loaded_model.load_state_dict(torch.load(PATH))

#학습모드가 아닌 eval모드로 전환필수
loaded_model.eval()

print("기존 모델 로드 완료.")

In [22]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output = torch.tensor([START_TOKEN]).to(device)

    # 디코더의 예측 시작
    model.eval()
    for i in range(MAX_LENGTH):
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).to(device)

        src_padding_mask = gen_attention_mask(input).to(device)
        tgt_padding_mask = gen_attention_mask(output).to(device)

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(END_TOKEN[0])):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.to(device)], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [27]:
result = predict("놀고싶다")
result = predict("배고파")
result = predict("고민 상담 해줘")
#매번 잘 나오지는 않지만 일부 잘 만드는 경우가 있다.

Input: 놀고싶다
Output: 좋아하는 마음이 복잡할 때은 항상 막히는 .
Input: 배고파
Output: 뭐라도 드세요 .
Input: 고민 상담 해줘
Output: 네 말씀해주세요 .


In [75]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from tqdm import tqdm

def predict(sentence, model, vocab):
    prediction = evaluate(sentence, model, vocab)
    # vocab.Decode의 인자는 정수 리스트여야 하므로 map(int, ...)를 사용합니다.
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))
    return predicted_sentence


def evaluate(sentence, model, vocab):
    sentence = preprocess_sentence(sentence)
    input_tensor = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output_tensor = torch.tensor([START_TOKEN]).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(MAX_LENGTH):
            src_mask = model.generate_square_subsequent_mask(input_tensor.shape[1]).to(device)
            tgt_mask = model.generate_square_subsequent_mask(output_tensor.shape[1]).to(device)

            src_padding_mask = gen_attention_mask(input_tensor).to(device)
            tgt_padding_mask = gen_attention_mask(output_tensor).to(device)

            predictions = model(input_tensor, output_tensor, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
            predictions = predictions[:, -1:, :]
            predicted_id = torch.argmax(predictions, axis=-1)

            # if torch.equal(predicted_id[0,0], torch.tensor(END_TOKEN[0])):
            #     break
            # predicted_id[0,0] 텐서에서 숫자 값을 꺼내 파이썬 정수와 직접 비교
            if predicted_id[0,0].item() == END_TOKEN[0]:
                break
            output_tensor = torch.cat([output_tensor, predicted_id], axis=1)

    return torch.squeeze(output_tensor, axis=0).cpu().numpy()


# --- [BLEU 점수 계산 함수] ---
def calculate_bleu_score(model, questions_corpus, answers_corpus, vocab, weights):
    """
    Args:
        model (torch.nn.Module): 평가할 학습된 모델
        questions_corpus (list): 토큰화된 질문 문장 리스트
        answers_corpus (list): 토큰화된 정답 문장 리스트
        vocab (spm.SentencePieceProcessor): SentencePiece 토크나이저
    """
    print("BLEU 점수 계산을 시작합니다...")
    
    references = []
    hypotheses = []
    
    # SmoothingFunction은 reference에 없는 단어가 hypothesis에 나올 경우 점수가 0이 되는 것을 방지합니다.
    smoothie = SmoothingFunction().method1

    for que_tokens, ans_tokens in tqdm(zip(questions_corpus, answers_corpus), total=len(questions_corpus)):
        # 정답 문장은 이중 리스트 형태(하나의 질문에 여러 정답이 있을 수 있기 때문)
        references.append([ans_tokens])
        
        # 질문 토큰을 다시 문자열로 합쳐 모델의 입력으로 사용
        question_str = vocab.decode(que_tokens)
        
        # 모델 예측
        predicted_sentence = predict(question_str, model, vocab)
        
        # 예측된 문장을 다시 토큰화
        hypothesis_tokens = vocab.encode_as_pieces(predicted_sentence)
        hypotheses.append(hypothesis_tokens)

    # corpus_bleu를 사용하여 전체 테스트셋에 대한 BLEU 점수 계산
    # weights는 BLEU-1, 2, 3, 4의 가중치를 의미하며, (0.25, 0.25, 0.25, 0.25)는 BLEU-4를 의미합니다.
    bleu_score = corpus_bleu(references, hypotheses, weights, smoothing_function=smoothie)
    
    return bleu_score



# --- [함수 실행] ---
#평가에는 test 데이터셋을 이용
# bleu_score = calculate_bleu_score(model, que_corpus_test, ans_corpus_test, vocab)
# print(f"\n모델의 BLEU-4 점수: {bleu_score:.4f}")
# print(f"모델의 BLEU-4 점수 (백분율): {bleu_score*100:.2f}%")

In [76]:
#데이터가 짧은 대화 위주다 보니 n이 높은것보단 낮은게 더 평가점수가 높을것이라 예상 
#이를 검증하는 간단한 실험 진행
# 테스트하고 싶은 weights를 딕셔너리로 정의
weights_to_test = {
    "BLEU-1 (Individual)": (1, 0, 0, 0),
    "BLEU-2 (Cumulative)": (0.5, 0.5, 0, 0),
    "BLEU-3 (Cumulative)": (1/3, 1/3, 1/3, 0),
    "BLEU-4 (Cumulative)": (0.25, 0.25, 0.25, 0.25)
}

print("다양한 가중치에 따른 BLEU SCORE 평가")
print("-" * 50)

for name, weights in weights_to_test.items():
    score = calculate_bleu_score(model, que_corpus_test, ans_corpus_test, vocab, weights)
    print(f"{name}: {score*100:.2f}%")
    print("-" * 50)

다양한 가중치에 따른 BLEU SCORE 평가
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 28.85it/s]


BLEU-1 (Individual): 35.96%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 28.89it/s]


BLEU-2 (Cumulative): 25.93%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:07<00:00, 28.84it/s]


BLEU-3 (Cumulative): 21.52%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 28.97it/s]


BLEU-4 (Cumulative): 18.58%
--------------------------------------------------


예상에 맞게 낮은 n에 가중치를 주는편이 그렇지 않은쪽보다 더 좋은 점수를 얻음

In [77]:
#단순 그램별 스코어 차이도 확인

weights_to_test = {
    "BLEU-1 (Individual)": (1, 0, 0, 0),
    "BLEU-2 (Cumulative)": (0, 1, 0, 0),
    "BLEU-3 (Cumulative)": (0, 0, 1, 0),
    "BLEU-4 (Cumulative)": (0, 0, 0, 1)
}

print("다양한 가중치에 따른 BLEU SCORE 평가")
print("-" * 50)

for name, weights in weights_to_test.items():
    score = calculate_bleu_score(model, que_corpus_test, ans_corpus_test, vocab, weights)
    print(f"{name}: {score*100:.2f}%")
    print("-" * 50)

다양한 가중치에 따른 BLEU SCORE 평가
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 28.90it/s]


BLEU-1 (Individual): 35.96%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 28.88it/s]


BLEU-2 (Cumulative): 18.70%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 29.08it/s]


BLEU-3 (Cumulative): 14.81%
--------------------------------------------------
BLEU 점수 계산을 시작합니다...


100%|██████████| 1933/1933 [01:06<00:00, 29.12it/s]


BLEU-4 (Cumulative): 11.96%
--------------------------------------------------
